In [1]:
%%HTML
<style type="text/css">
table.dataframe td, table.dataframe th {
    border: 1px  black solid !important;
  color: black !important;
}
</style>

In [2]:
from bs4 import BeautifulSoup
!python -V

Python 3.6.8 :: Anaconda, Inc.


In [3]:
#$$$ 1 Input file names and imports
import subprocess
from bs4 import BeautifulSoup,sys,os, re
import pandas as pd
import numpy as np
pd.options.display.max_columns = None

#Specify path of sentence:
tmp_path=os.getenv('HOME_anu_tmp')+'/tmp/'
# eng_file_name = 'cc_conjE'
eng_file_name = 'BUgol2.1E'

sent_no = '2.32' #2.29, 2.21, 2.61, 2.14, 2.64
path_tmp= tmp_path + eng_file_name + "_tmp/" + sent_no

#------------------------------------------------------------------------------------
hfilename = path_tmp +  '/H_wordid-word_mapping.dat'
efilename = path_tmp + '/E_wordid-word_mapping.dat'
efilename_alternate = path_tmp + '/word.dat'

hparserid_to_wid = path_tmp + '/H_parserid-wordid_mapping.dat'
nandani_file = path_tmp +  '/corpus_specific_dic_facts_for_one_sent.dat'
roja_transliterate_file = path_tmp +  '/Roja_chk_transliterated_words.dat'
# roja_transliterate_file = path_tmp +  '/results_of_transliteration.dat'
html_file = path_tmp +'/'+ eng_file_name +'_table1.html'
log_file = path_tmp + '/log_htmltocsv'
if os.path.exists(log_file):
    os.remove(log_file)
log = open(log_file,'a')
#------------------------------------------------------------------------------------

In [4]:
#$$$ 2 func is to replace manju mam id's to word ids of hindi assigned words below every eng wordid

def func(x):
    col = x.tolist()
    new_col=[];all_series=[];final_cell_value=""
    #if(str('   anu_exact_match  ', 'utf-8')) in col:
    #if unicode('   anu_exact_match  ') in col:
    if '   anu_exact_match  ' in col:
        new_col =[str(i) for i in col]
    else:
        for count, i in enumerate(col,0):
            if(i!= ' .'):
                change = i.lstrip().rstrip()
                if '/' in change:
                    possibilities = [str(i.lstrip().rstrip()) for i in change.split("/")]
                    new2=[]
                    for poss in possibilities:
                        if '+' in poss:
                            g = poss.split('+')
                            new1=[]
                            for one in g:
#                                 print("((one))",one, type(one))
                                new1.append(p2w[int(one)])
                            final_cell_value1 = " ".join(new1)
                            new2.append(final_cell_value1)
                        else:
#                             print("***poss", poss, type(poss))
                            final_cell_value2 = p2w[int(poss)]
                            new2.append(final_cell_value2)
                    final_cell_value = "/".join(new2)
                    new_col.append(final_cell_value)
                    
                elif '+' in change:
                    grouping = [str(i.lstrip().rstrip()) for i in change.split("+")]
                    new=[]
#                     print(p2w)
                    for g in grouping:
#                         print(g)
                        new.append(p2w[int(g)])
                    final_cell_value = " ".join(new)
                    new_col.append(final_cell_value)
                else:
                    final_cell_value = p2w[int(change)]
                    new_col.append(final_cell_value)
            else:
                final_cell_value = '0'
                new_col.append(final_cell_value)
    new_col.append(final_cell_value)
    new_col = new_col[:-1]
    new_x = pd.Series(new_col)
#     print(new_x)
    return(new_x)


In [5]:
#$$$ 3
#Function to extract dictionary from H_wordid-word_mapping.dat
def create_dict(filename,string):
    with open(filename,"r") as f1: 
        text = f1.read().split("\n")
        while("" in text) :
            text.remove("")
        p2w = {}
        for line in text:
            t = line.lstrip(string).strip(')').split("\t")
            p2w[int(t[1].lstrip("P"))] = t[2]
    return(p2w)


#Extract nandani_mapping dictionary[eids to hids] from corpus_specific_dic_facts_for_one_sent.dat
def extract_dictionary_from_deftemplate(filename):
    with open(filename, "r") as f:
        data = f.read().split("\n")
#         print(data)
        while "" in data:
            data.remove("")
        tranliterate_dict={}
        for line in data:
#             print(line)
            key = line.split(")")[0].lstrip("Edict-Hdict (E_id ")
            val = line.split(")")[2].lstrip("(H_id ")
#             print(key, val)
            tranliterate_dict[key]=val
    return(tranliterate_dict)

#given a value returns corresponding key from a dictionary
def return_key_from_value(dictionary, value):
    key =[ k for k,v in dictionary.items() if v == value]
    return (key[0])

#created for extracting eid-hid pair corresponding to eword-hword pair of roja_chk_transliteration_out
def get_E_H_dict_Ids(filename):
    tmp={}
    with open(filename,'r') as f:
        
#         print(f.read().strip("\n").split("\n"))
        entries=f.read().strip("\n").split("\n")
#         print("=>",entries)
        entries = [item.rstrip(")") for item in entries]
#         print(h2w)
        for entry in entries:
#             print(entry)
            eword = entry.split("\t")[1]
            hword = entry.split("\t")[2]
#             print(eword, hword)
            eid = return_key_from_value(e2w, eword)
            hid = return_key_from_value(h2w, hword)
#             print(eid, hid)
            tmp[str(eid)] = str(hid)
    return(tmp)    

#------------------------------------------------------------------------------------


In [6]:
#$$$ 4

#Functions for anchor calculations:


def union_of_columns_verically(dfs):
    final_row_in_csv=[]
    for j in range(0,no_of_eng_words):
        hindi_allocations_list=[]
        #for every row extracted all non zero entries and stored in hindi_allocations_list
        hindi_allocations_list = [str(i) for i in dfs.iloc[:,j].tolist() if i!='0' and i!=0]
    
        #For an empty hindi_allocations_list i.e verical column equivalent to 1 eng id appended '0' entry in it.
        if not hindi_allocations_list:
            hindi_allocations_list.append('0')
            
        #hindi_allocations string contains all entries in vertical column with "#" seperator.
        hindi_allocations = "#".join([str(i) for i in hindi_allocations_list])        
        final_row_in_csv.append(hindi_allocations)
    return(final_row_in_csv)

def remove_duplicates_from_union(final_row_in_csv):
    anchor1=[]; anchor1_str_list=[]; count_dict_list=[]
    for j in range(0,no_of_eng_words):
        temp=[];temp1=[]; temp_count_dict={}
        temp = re.split('#|/',final_row_in_csv[j])
        temp1 = list(dict.fromkeys(re.split('#|/',final_row_in_csv[j])))
        count_info = [temp.count(x) for x in temp1]

        for i in range(len(temp1)):
            temp_count_dict[temp1[i]] = count_info[i]
    
        count_dict_list.append(temp_count_dict)
        anchor1.append(temp1)
        anchor1_str_list.append("#".join(temp1))  
    return(anchor1, anchor1_str_list, count_dict_list)

def remove_multi_entry_in_a_cell(anchor1):
    anchor2=[]; anchor2_str_list=[]
    for j in range(0,no_of_eng_words):
        if len(anchor1[j]) == 1:
            temp = anchor1[j]
        else:
            temp=['0']
        anchor2.append(temp)
        anchor2_str_list.append("".join(temp))
#         print(anchor1[j],"=>", anchor2[j])
    return(anchor2, anchor2_str_list)

def remove_single_hindi_id_in_more_than_one_column(anchor2_str_list):
    anchor3_str_list=anchor2_str_list
    
    repetated_entries=[]
    for i in range(0, len(anchor2_str_list)):
    
#     print(anchor2_str_list[i], anchor2_str_list.count(anchor2_str_list[i]))
        if anchor2_str_list.count(anchor2_str_list[i]) > 1:
            repetated_entries.append(anchor2_str_list[i])
            anchor3_str_list[i] = '0'
#     print(anchor2_str_list)
#     print(anchor3_str_list)
#     print(repetated_entries)
    for j in range(0, len(anchor3_str_list)):
        for i in range(0,len(repetated_entries)):
#         print(repetated_entries[i])
            if repetated_entries != '0':
                if repetated_entries[i] == anchor3_str_list[j]:
                    anchor3_str_list[j] = '0'
    return(anchor3_str_list)


#Function used in visualization
def replace_id_by_id_word_pair_for_visualization(unique):
    show_hindi[0]='0'
    unique_words = []
    for i in range(0,len(unique)):
        temp=[]
        if i== 0:
            unique_words.append(unique[i])
        if i > 0:
            check_multiple = unique[i].split(" ")
            if len(check_multiple) == 1:
                temp.append(show_hindi[int(unique[i])])
            else:
#         if len(check_multiple) > 1:
                for item in check_multiple:
                    temp.append(show_hindi[int(item)])
#         print(temp)
            unique_words.append(" ".join(temp))
    return(unique_words) 

In [7]:
#$$$ 5

##CREATION OF PYTHON DICTS and LISTS FROM NECESSARY FILES, csv,sys,os
try:   
    h2w = create_dict(hfilename, '(H_wordid-word')
#     print(h2w)
    show_hindi ={}    
    for k,v in h2w.items():
        show_hindi[k] = str(k)+"_"+v
#     print(show_hindi)
    hin = [show_hindi[i] for i in sorted(show_hindi.keys())]
    hindi_word = list(show_hindi.values())

except:
    print("FILE MISSING: " + hfilename )
    log.write("FILE MISSING: " + hfilename + "\n")
    
#-----------------------------------------------------------------------------------------
try:
    p2w = create_dict(hparserid_to_wid, '(H_parserid-wordid')
except:
    print("*FILE MISSING: " + hparserid_to_wid)
    log.write("FILE MISSING: " + hparserid_to_wid+ "\n")


try:
    html = open(html_file).read()
    soup = BeautifulSoup(html, "lxml")
    table = soup.find('table')
    table_rows = table.find_all('tr')
    l=[]
#     print(table_rows)
    for tr in table_rows:
        td = tr.find_all('td')
        row = [tr.text for tr in td] 
        l.append(row)
    df = pd.DataFrame(l)
    final = df[9:-1].drop(df.columns[[-1]], axis=1)
    final=final[0:23]
    final.to_csv(path_tmp +'/'+ eng_file_name + "_"+sent_no + ".csv", index=False)
    
except:
    print("FILE MISSING: " + html_file)
    log.write("FILE MISSING: " + html_file+ "\n")
    
    
    

try:
    new = final.apply(func)
#     print(new)
#     if new==0:
#         break
#     print("************",new)
    new.to_csv(path_tmp +'/'+ eng_file_name + "_"+sent_no + "_1.csv", index=False)
except :
    print("Discrepance in hindi manju id, parser id and word id")
    log.write("Discrepance in hindi manju id, parser id and word id" + "\n")
    log.close()
    sys.exit()
    
    
# if new=='exit':
#     sys.exit()
# else:
    


#------------------------------------------------------------------------------------
try:
#     print(nandani_file)
    data="";
    nandani_mapping = extract_dictionary_from_deftemplate(nandani_file)  
#     print(nandani_mapping)
except:
    print("FILE MISSING: " + nandani_mapping )
    log.write("FILE MISSING: " + nandani_mapping + "\n")

#------------------------------------------------------------------------------------

try:
    # extracting df from BUgol2.1E_2.21_1.csv which contains  old dictionary facts with hindi word is and not parser id   
    dfs = pd.read_csv(path_tmp +'/'+ eng_file_name + "_"+sent_no + "_1.csv")
    #row index started from 1 instead of 0, which was earlier.
    dfs.index = np.arange(1,len(dfs)+1)
    # print(dfs.shape)
    # r = len(h2w)
    # c = dfs.shape[1] - 1
    # print(r, c)
    # r_list = range(1,r+1)
    # c_list =range(1,c+1)
    # print(r_list, c_list)
    # df = pd.Dataframe(rows=r_list, columns = c_list)
    # df = pd.DataFrame(index=r_list, columns = c_list)
    # print(df)
    #Creation of resources list from 1st column of dataframe
    resources = [i.lstrip().rstrip() for i in dfs.iloc[:, 0].tolist()]
    #Creation of resources_dict which mapping of alphabets to big names from 1st column of dataframe
    letters = [chr(i) for i in range(65, 88)]
    resource_dict={}
    for k,v in zip(letters,resources):
        resource_dict[k]=v
    # display(dfs)
    # resource_dict_invert= {v: k for k, v in resource_dict.items()}
    no_of_eng_words = dfs.shape[1]

except:
    print("FILE MISSING: " + path_tmp +'/'+ eng_file_name + "_"+sent_no + "_1.csv" )
    log.write("FILE MISSING: " + path_tmp +'/'+ eng_file_name + "_"+sent_no + "_1.csv" + "\n")

#------------------------------------------------------------------------------------
try:
    e2w = create_dict(efilename, '(E_wordid-word')
    #     print(e2w)
    show_eng ={}    
    for k,v in e2w.items():
        show_eng[k] = str(k)+"_"+v
    #     print(show_eng)
    eng = [show_eng[i] for i in sorted(show_eng.keys())]
#     print(show_eng.values(), type(show_eng.values()))
    title=["0"]+list(show_eng.values())#.insert(0,'0')

        
except:
    print("FILE MISSING: " + efilename )
    log.write("FILE MISSING: " + efilename+ "\n")
    command = "awk '{printf $3}' "+efilename_alternate
    print(command)
    x=subprocess.Popen(command,stdin=subprocess.PIPE,stdout=subprocess.PIPE, shell=True).stdout.read()
#     x1 = x.decode(encoding="utf-8", errors="strict")
    x1=x.decode("utf-8") 
    x2 = x1.split(")")
    while "" in x2:
        x2.remove("")
    print(x2, len(x2), type(x2))
    title=["0"] + list(x2[:-1])
    print(type(title))
    print(title, len(title), no_of_eng_words)
    print(dfs.shape)
#     print("===>",x,"\n", str(x))    

try:
    tranliterate_dict={}
    transliterate_mapping = get_E_H_dict_Ids(roja_transliterate_file)  
except:
    print("FILE MISSING: " + roja_transliterate_file )
    log.write("FILE MISSING: " + roja_transliterate_file + "\n")

# display(dfs)
hindi_wordnet = list(dfs.iloc[13])
partial_match = list(dfs.iloc[15])
dfs.drop(13, inplace=True)
dfs.drop(15, inplace=True)
# print(hindi_wordnet)
# print(partial_match)
# dfs

In [8]:
#$$$ 6
#Roja K layer partial match
try:
    
    k_layer_ids_file = path_tmp + '/H_alignment_parserid.csv'
    with open(k_layer_ids_file, 'r') as f:
        k_layer_ids = f.read().split("\n")[1].strip("\r").split(",")
    # print(k_layer_ids)
    for n, i in enumerate(k_layer_ids):
           if i == '-':
                k_layer_ids[n] = 0
    for n, i in enumerate(k_layer_ids):
           if i == '_':
                k_layer_ids[n] = 0

#     print(k_layer_ids, len(k_layer_ids), no_of_eng_words)

    dfs.loc[24] = k_layer_ids
    # dfs
except:
    print("FILE MISSING: " + k_layer_ids_file )
    log.write("FILE MISSING: " + k_layer_ids_file + "\n")
dfs

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26
1,anu_exact_match,0,0,0,0,0,0,0,0,0,0,0,0,13,0,0,0,0,10 11 12,0,0,0,0,0,0,0,0
2,anu_exact_match_without_vib,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,anu_root_match,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,anu_root_match_without_vib,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,multi_dictionary_match,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,dictionary_match,0,0,0,0,0,0,0,0,9,0,0,0,0,0,0,0,0,0,0,0,0,25,0,20,21/22,23 24
7,dictionary_match_without_vib,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,multi_hindi_wordnet_match,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,hindi_wordnet_match,0,0,0,0,0,0,0,0,0,0,0,0,13,0,0,0,0,0,0,0,0,25,0,20,7/21/22,23 24
10,single_verb_match,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [9]:
#$$$ 7

# Calling function for Union of columns and removing duplicates:

final_row_in_csv = union_of_columns_verically(dfs)
final_row_in_csv[0]="Verical Union v1"
print(final_row_in_csv)
anchor1, anchor1_str_list, count_dict_list = remove_duplicates_from_union(final_row_in_csv)
anchor1[0] = 'Potential anchors v1' 
anchor1_str_list[0] = 'Potential anchors v1' 
print(anchor1_str_list)
anchor2, anchor2_str_list = remove_multi_entry_in_a_cell(anchor1)
print(anchor2_str_list)
anchor3_str_list = remove_single_hindi_id_in_more_than_one_column(anchor2_str_list)
anchor3_str_list[0]="Starting anchor v1"
print(anchor3_str_list)
dfs.loc[25] = anchor1_str_list
dfs.loc[26] = anchor3_str_list
dfs

['Verical Union v1', '0', '0', '0', '0', '0', '0', '0', '2/4/12/15/18', '9', '0', '0', '0', '13#13#13', '2/4/12/15/18', '0', '0', '0', '10 11 12#10 11 12', '0', '0', '0', '25#25#6', '0', '20#20', '21/22#7/21/22', '23 24#23 24']
['Potential anchors v1', '0', '0', '0', '0', '0', '0', '0', '2#4#12#15#18', '9', '0', '0', '0', '13', '2#4#12#15#18', '0', '0', '0', '10 11 12', '0', '0', '0', '25#6', '0', '20', '21#22#7', '23 24']
['0', '0', '0', '0', '0', '0', '0', '0', '0', '9', '0', '0', '0', '13', '0', '0', '0', '0', '10 11 12', '0', '0', '0', '0', '0', '20', '0', '23 24']
['Starting anchor v1', '0', '0', '0', '0', '0', '0', '0', '0', '9', '0', '0', '0', '13', '0', '0', '0', '0', '10 11 12', '0', '0', '0', '0', '0', '20', '0', '23 24']


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26
1,anu_exact_match,0,0,0,0,0,0,0,0,0,0,0,0,13,0,0,0,0,10 11 12,0,0,0,0,0,0,0,0
2,anu_exact_match_without_vib,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,anu_root_match,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,anu_root_match_without_vib,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,multi_dictionary_match,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,dictionary_match,0,0,0,0,0,0,0,0,9,0,0,0,0,0,0,0,0,0,0,0,0,25,0,20,21/22,23 24
7,dictionary_match_without_vib,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,multi_hindi_wordnet_match,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,hindi_wordnet_match,0,0,0,0,0,0,0,0,0,0,0,0,13,0,0,0,0,0,0,0,0,25,0,20,7/21/22,23 24
10,single_verb_match,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [10]:
#$$$ 8
#extracting Roja and Nandani Dictionary values and inserting in csv
try:
    ## print(transliterate_mapping)
    roja_transliterate_list=[]; 
    for j in range(0,no_of_eng_words):
        if str(j) in transliterate_mapping.keys():
    #         print(str(j), transliterate_mapping[str(j)])
            roja_transliterate_list.append(transliterate_mapping[str(j)])
        else:
    #         print(str(j), '0')
            roja_transliterate_list.append('0')
    roja_transliterate_list[0] = 'Roja Transliterate'
    # print(roja_transliterate_list)   
    dfs.loc[27] = roja_transliterate_list
except:
    print("FILE MISSING: " + roja_transliterate_file )
    log.write("FILE MISSING: " + roja_transliterate_file  + "\n")


try:
    # print(nandani_mapping)
    nandani_mapping_list=[]; 
    for j in range(0,no_of_eng_words):
        if str(j) in nandani_mapping.keys():
    #         print(str(j), transliterate_mapping[str(j)])
            nandani_mapping_list.append(nandani_mapping[str(j)])
        else:
    #         print(str(j), '0')
            nandani_mapping_list.append('0')

        #This is temp module which has nandani's eng_multi to hindi_multi mapping
        #Eg. {'10 11': '3 4'} for group of 10 and 11 hindi has 3 and 4.
        #TODO FUTYRE: We need to add one more layer showing grouping information 
        for every_entry in nandani_mapping.keys():
            if " " in every_entry:
                eng_group_list = every_entry.split(" ")
                hindi_group_list = nandani_mapping[every_entry].split(" ")
    #             print(eng_group_list, hindi_group_list)

                #Till now nandani's entries are of same length
                for i in range(0, len(eng_group_list)):
                    nandani_mapping[eng_group_list[i]]= hindi_group_list[i]
    #             print("Multiword entry in nandani dictionary")
    # print(nandani_mapping)
    nandani_mapping_list[0] = 'Nandani dict'
    dfs.loc[28] = nandani_mapping_list
except:
    print("FILE MISSING: " + nandani_file ) 
    log.write("FILE MISSING: " + nandani_file  + "\n")

dfs


FILE MISSING: /home/kishori/a/tmp_anu_dir/tmp/BUgol2.1E_tmp/2.32/corpus_specific_dic_facts_for_one_sent.dat


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26
1,anu_exact_match,0,0,0,0,0,0,0,0,0,0,0,0,13,0,0,0,0,10 11 12,0,0,0,0,0,0,0,0
2,anu_exact_match_without_vib,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,anu_root_match,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,anu_root_match_without_vib,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,multi_dictionary_match,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,dictionary_match,0,0,0,0,0,0,0,0,9,0,0,0,0,0,0,0,0,0,0,0,0,25,0,20,21/22,23 24
7,dictionary_match_without_vib,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,multi_hindi_wordnet_match,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,hindi_wordnet_match,0,0,0,0,0,0,0,0,0,0,0,0,13,0,0,0,0,0,0,0,0,25,0,20,7/21/22,23 24
10,single_verb_match,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [11]:
#$$$ 9
final_row_in_csv = union_of_columns_verically(dfs)
final_row_in_csv[0]="Verical Union v2"
print(final_row_in_csv)
anchor1, anchor1_str_list, count_dict_list = remove_duplicates_from_union(final_row_in_csv)
anchor1[0] = 'Potential anchors v2' 
anchor1_str_list[0] = 'Potential anchors v2' 
print(anchor1_str_list)
anchor2, anchor2_str_list = remove_multi_entry_in_a_cell(anchor1)
print(anchor2_str_list)
anchor3_str_list = remove_single_hindi_id_in_more_than_one_column(anchor2_str_list)
anchor3_str_list[0]="Starting anchor v2"
print(anchor3_str_list)
dfs.loc[29] = anchor1_str_list
dfs.loc[30] = anchor3_str_list
dfs.to_csv(path_tmp +'/'+ eng_file_name + "_"+sent_no + "_2.csv", index=False)
dfs

['Verical Union v2', '0', '0', '0', '0', '0', '0', '1', '2/4/12/15/18#2#4#12#15#18', '9#9#9', '0', '0', '0', '13#13#13#13#13', '2/4/12/15/18#2#4#12#15#18', '0', '0', '0', '10 11 12#10 11 12#10 11 12#10 11 12', '0', '0', '0', '25#25#6#25#6', '0', '20#20#20#20', '21/22#7/21/22#21#22#7', '23 24#23 24#23 24#23 24']
['Potential anchors v2', '0', '0', '0', '0', '0', '0', '1', '2#4#12#15#18', '9', '0', '0', '0', '13', '2#4#12#15#18', '0', '0', '0', '10 11 12', '0', '0', '0', '25#6', '0', '20', '21#22#7', '23 24']
['0', '0', '0', '0', '0', '0', '0', '1', '0', '9', '0', '0', '0', '13', '0', '0', '0', '0', '10 11 12', '0', '0', '0', '0', '0', '20', '0', '23 24']
['Starting anchor v2', '0', '0', '0', '0', '0', '0', '1', '0', '9', '0', '0', '0', '13', '0', '0', '0', '0', '10 11 12', '0', '0', '0', '0', '0', '20', '0', '23 24']


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26
1,anu_exact_match,0,0,0,0,0,0,0,0,0,0,0,0,13,0,0,0,0,10 11 12,0,0,0,0,0,0,0,0
2,anu_exact_match_without_vib,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,anu_root_match,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,anu_root_match_without_vib,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,multi_dictionary_match,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,dictionary_match,0,0,0,0,0,0,0,0,9,0,0,0,0,0,0,0,0,0,0,0,0,25,0,20,21/22,23 24
7,dictionary_match_without_vib,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,multi_hindi_wordnet_match,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,hindi_wordnet_match,0,0,0,0,0,0,0,0,0,0,0,0,13,0,0,0,0,0,0,0,0,25,0,20,7/21/22,23 24
10,single_verb_match,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [12]:
#$$$ 10

pd.options.display.max_columns = None
pd.set_option('display.max_colwidth',-1)
potentialv1 = dfs.loc[25].to_list()
new_dfs = pd.DataFrame([potentialv1])

try:
    startingv1 = dfs.loc[26].to_list()
#     print(unique)
    startingv1_words = replace_id_by_id_word_pair_for_visualization(startingv1)
#     print(unique_words)
    new_dfs = new_dfs.append(pd.Series(startingv1_words, index=new_dfs.columns ), ignore_index=True)

except:
    print("Starting anchor v2 has no results")
    log.write("Starting anchor v2 has no results to write in short"  + "\n")


try:
    roja = dfs.loc[27].to_list()
#     print(roja)
    roja_words = replace_id_by_id_word_pair_for_visualization(roja)
#     print(roja_words)
    new_dfs = new_dfs.append(pd.Series(roja_words, index=new_dfs.columns ), ignore_index=True)

except:
    print("Roja transliterate has no results")
    log.write("Roja transliterate has no results to write in short"  + "\n")

try:
    nandani = dfs.loc[28].to_list()
#     print(nandani)
    nandani_words = replace_id_by_id_word_pair_for_visualization(nandani)
#     print(nandani_words)
    new_dfs = new_dfs.append(pd.Series(nandani_words, index=new_dfs.columns ), ignore_index=True)

except:
    print("Nandani transliterate has no results")
    log.write("Roja transliterate has no results to write in short"  + "\n")
    
potentialv2 = dfs.loc[29].to_list()
new_dfs = new_dfs.append(pd.Series(potentialv2, index=new_dfs.columns ), ignore_index=True)


try:
    startingv2 = dfs.loc[30].to_list()
#     print(startingv2)
    startingv2_words = replace_id_by_id_word_pair_for_visualization(startingv2)
#     print(startingv2_words)
    new_dfs = new_dfs.append(pd.Series(startingv2_words, index=new_dfs.columns ), ignore_index=True)

except:
    print("Starting anchor v2 has no results")
    log.write("Roja transliterate has no results to write in short" + "\n")
print(title)
new_dfs.columns =title
hindi_row = [" | ".join(hindi_word)] + ['.'] * (new_dfs.shape[1]-1)


new_dfs = new_dfs.append(pd.Series(hindi_row, index=new_dfs.columns ), ignore_index=True)

new_dfs.to_csv(path_tmp +"/short.csv", index=False)
new_dfs.to_html(path_tmp + '/short.html')

new_dfs



Nandani transliterate has no results
['0', '1_The', '2_Himalayan', '3_uplift', '4_out', '5_of', '6_the', '7_Tethys', '8_sea', '9_and', '10_subsidence', '11_of', '12_the', '13_northern', '14_flank', '15_of', '16_the', '17_peninsular', '18_plateau', '19_resulted', '20_in', '21_the', '22_formation', '23_of', '24_a', '25_large', '26_basin']


,0,1_The,2_Himalayan,3_uplift,4_out,5_of,6_the,7_Tethys,8_sea,9_and,10_subsidence,11_of,12_the,13_northern,14_flank,15_of,16_the,17_peninsular,18_plateau,19_resulted,20_in,21_the,22_formation,23_of,24_a,25_large,26_basin
0,Potential anchors v1,0,0,0,0,0,0,0,2#4#12#15#18,9,0,0,0,13,2#4#12#15#18,0,0,0,10 11 12,0,0,0,25#6,0,20,21#22#7,23 24
1,Starting anchor v1,0,0,0,0,0,0,0,0,9_waWA,0,0,0,13_uwwarI,0,0,0,0,10_prAyaxvIpIya 11_paTAra 12_ke,0,0,0,0,0,20_eka,0,23_xroNI 24_kA
2,Roja Transliterate,0,0,0,0,0,0,1_teWisa,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Potential anchors v2,0,0,0,0,0,0,1,2#4#12#15#18,9,0,0,0,13,2#4#12#15#18,0,0,0,10 11 12,0,0,0,25#6,0,20,21#22#7,23 24
4,Starting anchor v2,0,0,0,0,0,0,1_teWisa,0,9_waWA,0,0,0,13_uwwarI,0,0,0,0,10_prAyaxvIpIya 11_paTAra 12_ke,0,0,0,0,0,20_eka,0,23_xroNI 24_kA
5,1_teWisa | 2_ke | 3_himAlaya | 4_ke | 5_rUpa | 6_meM | 7_Upara | 8_uTane | 9_waWA | 10_prAyaxvIpIya | 11_paTAra | 12_ke | 13_uwwarI | 14_kinAre | 15_ke | 16_nIce | 17_Xazsane | 18_ke | 19_pariNAmasvarUpa | 20_eka | 21_bahuwa | 22_badZI | 23_xroNI | 24_kA | 25_nirmANa | 26_huA,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.


In [13]:
log.close()